# Advanced tour of the Bayesian Optimization package

In [1]:
from bayes_opt import BayesianOptimization

## 1. Suggest-Evaluate-Register Paradigm

Internally the `maximize` method is simply a wrapper around the methods `suggest`, `probe`, and `register`. If you need more control over your optimization loops the Suggest-Evaluate-Register paradigm should give you that extra flexibility.

For an example of running the `BayesianOptimization` in a distributed fashion (where the function being optimized is evaluated concurrently in different cores/machines/servers), checkout the `async_optimization.py` script in the examples folder.

In [2]:
# Let's start by definying our function, bounds, and instanciating an optimization object.
def black_box_function(x, y):
    return -x ** 2 - (y - 1) ** 2 + 1

Notice that the evaluation of the blackbox function will NOT be carried out by the optimizer object. We are simulating a situation where this function could be being executed in a different machine, maybe it is written in another language, or it could even be the result of a chemistry experiment. Whatever the case may be, you can take charge of it and as long as you don't invoke the `probe` or `maximize` methods directly, the optimizer object will ignore the blackbox function.

In [3]:
optimizer = BayesianOptimization(
    f=None,
    pbounds={'x': (-2, 2), 'y': (-3, 3)},
    verbose=2,
    random_state=1,
)

One extra ingredient we will need is an `UtilityFunction` instance. In case it is not clear why, take a look at the literature to understand better how this method works.

In [4]:
from bayes_opt import UtilityFunction

utility = UtilityFunction(kind="ucb", kappa=2.5, xi=0.0)

The `suggest` method of our optimizer can be called at any time. What you get back is a suggestion for the next parameter combination the optimizer wants to probe.

Notice that while the optimizer hasn't observed any points, the suggestions will be random. However, they will stop being random and improve in quality the more points are observed.

In [5]:
next_point_to_probe = optimizer.suggest(utility)
print("Next point to probe is:", next_point_to_probe)

Next point to probe is: {'x': -0.331911981189704, 'y': 1.3219469606529488}


You are now free to evaluate your function at the suggested point however/whenever you like.

In [6]:
target = black_box_function(**next_point_to_probe)
print("Found the target value to be:", target)

Found the target value to be: 0.7861845912690542


Last thing left to do is to tell the optimizer what target value was observed.

In [7]:
optimizer.register(
    x=next_point_to_probe,
    y=target,
)

### 1.1 The maximize loop

And that's it. By repeating the steps above you recreate the internals of the `maximize` method. This should give you all the flexibility you need to log progress, hault execution, perform concurrent evaluations, etc.

In [8]:
for _ in range(5):
    next_point = optimizer.suggest(utility)
    target = black_box_function(**next_point)
    optimizer.register(x=next_point, y=target)
    
    print(target, next_point)
print(optimizer.max)

-18.49187152919165 {'x': 1.8861546000771092, 'y': -2.9917780942581977}
0.7911494590443674 {'x': -0.31764604716962586, 'y': 1.3285597809731806}
-6.738653892565658 {'x': -1.9777941453451842, 'y': 2.9562679292990444}
-6.741655559035455 {'x': 1.9742487076906352, 'y': 2.960611537050956}
-7.311459236889235 {'x': -1.999211910635848, 'y': -1.0771641661796976}
{'target': 0.7911494590443674, 'params': {'x': -0.31764604716962586, 'y': 1.3285597809731806}}


## 2. Dealing with discrete parameters

**There is no principled way of dealing with discrete parameters using this package.**

Ok, now that we got that out of the way, how do you do it? You're bound to be in a situation where some of your function's parameters may only take on discrete values. Unfortunately, the nature of bayesian optimization with gaussian processes doesn't allow for an easy/intuitive way of dealing with discrete parameters - but that doesn't mean it is impossible. The example below showcases a simple, yet reasonably adequate, way to dealing with discrete parameters.

In [9]:
def func_with_discrete_params(x, y, d):
    # Simulate necessity of having d being discrete.
    assert type(d) == int
    
    return ((x + y + d) // (1 + d)) / (1 + (x + y) ** 2)

In [10]:
def function_to_be_optimized(x, y, w):
    d = int(w)
    return func_with_discrete_params(x, y, d)

In [11]:
optimizer = BayesianOptimization(
    f=function_to_be_optimized,
    pbounds={'x': (-10, 10), 'y': (-10, 10), 'w': (0, 5)},
    verbose=2,
    random_state=1,
)

In [12]:
optimizer.maximize(alpha=1e-3)

|   iter    |  target   |     w     |     x     |     y     |
-------------------------------------------------------------
|  1        | -0.06199  |  2.085    |  4.406    | -9.998    |
|  2        | -0.0344   |  1.512    | -7.065    | -8.153    |
|  3        | -0.2177   |  0.9313   | -3.089    | -2.065    |
|  4        |  0.1865   |  2.694    | -1.616    |  3.704    |
|  5        | -0.2187   |  1.022    |  7.562    | -9.452    |
|  6        |  0.1868   |  2.533    | -1.728    |  3.815    |
|  7        |  0.04138  |  3.494    |  0.1725   |  6.707    |
|  8        |  0.0      |  4.737    | -3.817    |  4.586    |
|  9        |  0.1096   |  1.001    |  0.7072   |  3.447    |
|  10       |  0.04164  |  4.762    |  2.874    |  1.923    |
|  11       |  0.1892   |  0.04772  | -2.696    |  6.551    |
|  12       |  0.2199   |  0.02389  | -5.944    |  9.5      |
|  13       | -0.457    |  0.07146  | -9.618    |  7.259    |
|  14       |  0.1358   |  0.329    | -3.351    |  9.922    |
|  15   

## 3. Handling functions that may throw exceptions

We can handle probable exceptions in black-box functions in this framework by throwing `FuncFailed` exceptions. When the `FuncFailed` exception is raised, the value of this time's independent variable is discarded and has no influence on the ongoing fitting processes. Also, `OptimizationEvent.SKIP` event will be triggered.

In [13]:
import random
from bayes_opt.target_space import FuncFailed


def func_with_exception(x, y):
    if random.random() < 0.20:
        raise FuncFailed(x, y)  # 20% chance to throw exception
    return -x ** 2 - (y - 1) ** 2 + 1

In [14]:
optimizer = BayesianOptimization(
    f=func_with_exception,
    pbounds={'x': (2, 4), 'y': (-3, 3)},
    random_state=1,
    verbose=2,
)

In [15]:
optimizer.maximize(
    init_points=15,
    n_iter=30,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  1        | -7.135    |  2.834    |  1.322    |
|  2        | -7.78     |  2.0      | -1.186    |
|  3        | <skipped> |  2.294    | -2.446    |
|  4        | -8.341    |  2.373    | -0.9266   |
|  5        | -7.392    |  2.794    |  0.2329   |
|  6        | -7.069    |  2.838    |  1.111    |
|  7        | <skipped> |  2.409    |  2.269    |
|  8        | -3.223    |  2.055    |  1.023    |
|  9        | <skipped> |  2.835    |  0.3521   |
|  10       | -12.11    |  2.281    | -1.811    |
|  11       | -15.25    |  3.601    |  2.81     |
|  12       | <skipped> |  2.627    |  1.154    |
|  13       | <skipped> |  3.753    |  2.368    |
|  14       | -17.89    |  2.17     | -2.766    |
|  15       | -6.084    |  2.34     |  2.269    |
|  16       | -7.0      |  2.0      |  3.0      |
|  17       | -17.14    |  4.0      | -0.4635   |
|  18       | -3.505    |  2.0      |  0.2897   |


In [16]:
optimizer.max

{'target': -3.000000400520359, 'params': {'x': 2.0, 'y': 1.0006328667783022}}

## 4. Tuning the underlying Gaussian Process

The bayesian optimization algorithm works by performing a gaussian process regression of the observed combination of parameters and their associated target values. The predicted parameter$\rightarrow$target hyper-surface (and its uncertainty) is then used to guide the next best point to probe.

### 4.1 Passing parameter to the GP

Depending on the problemn it could be beneficial to change the default parameters of the underlying GP. You can simply pass GP parameters to the maximize method directly as you can see below:

In [17]:
optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds={'x': (-2, 2), 'y': (-3, 3)},
    verbose=2,
    random_state=1,
)
optimizer.maximize(
    init_points=1,
    n_iter=5,
    # What follows are GP regressor parameters
    alpha=1e-3,
    n_restarts_optimizer=5
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  1        |  0.7862   | -0.3319   |  1.322    |
|  2        | -18.49    |  1.886    | -2.992    |
|  3        |  0.7911   | -0.3176   |  1.329    |
|  4        | -6.173    | -1.796    |  2.987    |
|  5        | -2.834    |  1.502    |  2.256    |
|  6        | -4.831    | -1.997    | -0.3574   |


Another alternative, specially useful if you're calling `maximize` multiple times or optimizing outside the `maximize` loop, is to call the `set_gp_params` method.

In [18]:
optimizer.set_gp_params(normalize_y=True)

### 4.2 Tuning the `alpha` parameter

When dealing with functions with discrete parameters,or particularly erratic target space it might be beneficial to increase the value of the `alpha` parameter. This parameters controls how much noise the GP can handle, so increase it whenever you think that extra flexibility is needed.

### 4.3 Changing kernels

By default this package uses the Mattern 2.5 kernel. Depending on your use case you may find that tunning the GP kernel could be beneficial. You're on your own here since these are very specific solutions to very specific problems.

## Observers Continued

Observers are objects that subscribe and listen to particular events fired by the `BayesianOptimization` object. 

When an event gets fired a callback function is called with the event and the `BayesianOptimization` instance passed as parameters. The callback can be specified at the time of subscription. If none is given it will look for an `update` method from the observer.

In [19]:
from bayes_opt.event import OptimizationEvent

In [20]:
optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds={'x': (-2, 2), 'y': (-3, 3)},
    verbose=2,
    random_state=1,
)

In [21]:
class BasicObserver:
    def update(self, event, instance):
        """Does whatever you want with the event and `BayesianOptimization` instance."""
        print("Event `{}` was observed".format(event))

In [22]:
my_observer = BasicObserver()

optimizer.subscribe(
    event=OptimizationEvent.STEP,
    subscriber=my_observer,
    callback=None, # Will use the `update` method as callback
)

Alternatively you have the option to pass a completely different callback.

In [23]:
def my_callback(event, instance):
    print("Go nuts here!")

optimizer.subscribe(
    event=OptimizationEvent.START,
    subscriber="Any hashable object",
    callback=my_callback,
)

In [24]:
optimizer.maximize(init_points=1, n_iter=2)

Go nuts here!
Event `2` was observed
Event `2` was observed
Event `2` was observed


For a list of all default events you can checkout `OptimizationEvent.__members__`

In [25]:
OptimizationEvent.__members__

mappingproxy({'START': <OptimizationEvent.START: 1>,
              'STEP': <OptimizationEvent.STEP: 2>,
              'SKIP': <OptimizationEvent.SKIP: 3>,
              'END': <OptimizationEvent.END: 4>})